In [18]:
#  Hi, my name is Tate Doherty and this is my Barcode Scanner Project. This Barcode Scanner Project uses python in 
# Jupyter Notebooks that utilizes a webcam to scan barcodes and QR codes in real-time. It displays the scanned code's type and 
# data on a graphical user interface, stores the scan history in both CSV and XLSX files, and ensures that duplicate barcodes 
# are not included in the records. 

#importing statments and external libraries to successfully run the code
#p.s. you will most likely need to install PSimpleGUI to create a GUI for your camera application

#!pip install PySimpleGUI 
import cv2
import PySimpleGUI as sg
from pyzbar.pyzbar import decode
import numpy as np
import pandas as pd
import os
from datetime import datetime

# activating the camera
cap = cv2.VideoCapture(0)

if not cap is None:
    # creating a folder path for the "Barcodes Folder"
    folder_path = os.path.join(os.path.expanduser("~\Downloads"), "Barcodes Folder")

    # checking if the "Barcodes Folder" exists and creating it if needed
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # creating and naming the CSV and XLSX file paths
    csv_filename = 'barcodes_csv.csv'
    xlsx_filename = 'barcodes_xlsx.xlsx'
    csv_path = os.path.join(folder_path, csv_filename)
    xlsx_path = os.path.join(folder_path, xlsx_filename)

    # creating a DataFrame for the scanned barcodes
    if os.path.exists(csv_path):
        scanned_barcodes_df = pd.read_csv(csv_path)
    else:
        scanned_barcodes_df = pd.DataFrame(columns=['Barcode ID', 'Timestamp', 'Code Type', 'Code Data'])

    # starting the Barcode ID from the last index
    barcode_id = len(scanned_barcodes_df) + 1

    layout = [
        [sg.Image(filename='', key='-IMAGE-')],
        [sg.Text('Scanned Code Type:'), sg.Text('', key='-CODE-TYPE-')],
        [sg.Text('Scanned Data:'), sg.Text('', key='-DATA-')],
        [sg.Button('Exit')],
    ]

    window = sg.Window('Webcam Viewer', layout, finalize=True)

    export_message = None  
    
    # initializing with existing values
    unique_values = set(scanned_barcodes_df['Code Data']) 

    def export_data():
        # exporting barcode data to CSV and XLSX files
        scanned_barcodes_df.to_csv(csv_path, index=False)
        scanned_barcodes_df.to_excel(xlsx_path, index=False)

    while True:
        event, values = window.read(timeout=100)

        if event == sg.WIN_CLOSED or event == 'Exit':
            # exporting the data before exiting
            if export_message:
                export_data()
                print(export_message)
            break
        #returning error message if camera is not working
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame not read.")
            break

        # converting the frame to a format suitable for PySimpleGUI
        imgbytes = cv2.imencode('.png', frame)[1].tobytes()
        window['-IMAGE-'].update(data=imgbytes)

        # decoding barcodes/QR codes
        decoded_objects = decode(frame)

        if decoded_objects:
            first_code = decoded_objects[0]  
            code_type = first_code.type
            code_data = first_code.data.decode('utf-8')

            # checking if the barcode is unique and has not been scanned before
            if code_data not in unique_values:
                window['-CODE-TYPE-'].update(code_type)
                window['-DATA-'].update(code_data)

                # printing the information received from the webcam so the user can tell the barcode was scanned
                print("Code Type:", code_type)
                print("Decoded Data:", code_data)

                # creating a DataFrame for the new unique scanned barcode
                new_row = pd.DataFrame({'Barcode ID': [barcode_id], 'Timestamp': [datetime.now()],
                                        'Code Type': [code_type], 'Code Data': [code_data]})

                # concatenating the new row to the existing DataFrame
                scanned_barcodes_df = pd.concat([scanned_barcodes_df, new_row], ignore_index=True)

                # incrementing the Barcode ID
                barcode_id += 1

                # adding the code to the encountered values set
                unique_values.add(code_data)

    cap.release()
    window.close()

    # exporting data after closing the webcam viewer
    export_data()
    export_message = "Data exported to 'Downloads'. The files 'barcodes_csv and barcodes_xlsx' have been updated."

print(export_message)


Code Type: EAN13
Decoded Data: 9189781738034
Code Type: EAN13
Decoded Data: 9780486229270
Code Type: EAN13
Decoded Data: 0800759229277
Data exported to 'Downloads'. The files 'barcodes_csv and barcodes_xlsx' have been updated.
